<a href="https://www.kaggle.com/code/itsukikigoshi/super-rat?scriptVersionId=240428067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Check Python Version
import sys
import platform

print(sys.version)
print(platform.architecture()[0]) # Check whether 32 or 64bit

# Analysis Roadmap (逐次更新)
## Preconditions
- No mice in ocean
  - 海岸線の緯度と経度を得て，海岸線より海側のネズミ発生確率を0に
## Method
- Related Indicators
  - Temperature
  - Population density (mesh data?)
  - Altitude（∝1/temperature?）
  - Species（各ネズミの大まかな生息条件を調べられるとbetter）
  - Family Tree
- 簡単のために，X軸: 経度，Y軸: 緯度の2次元平面上で表現→この平面上に海岸線を描画できる？
- 数kmメッシュに区切って，各メッシュのネズミ発生確率を予測する
  - ただし，学習データ上でネズミの発生確率が0となる: limitation
  - もちろん海上のメッシュは発生確率0
  - 図を見る限り緯度経度0.2度メッシュくらいが現実的に解析可能そうなライン？

# Key Takeaways (逐次更新)
- .shpなど地理データをGeoDataFrameにする時は正しいCRS (Coordinate Reference System; 例えば[EPSG:3035](https://epsg.io/3035))の指定が肝要
- Anacondaでは，仮想環境にpipするのがbetter: conda installは使わない！!
- [主観]質の高い地理データはEUとUSに多い印象
- [主観]降水量などの時系列データは.nc (NetCDF)が多い

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Start Connection with SQLite
file_sqlite3 = "/kaggle/input/d/itsukikigoshi/openmice/OpenMICE.sqlite"
conn = sqlite3.connect(file_sqlite3)
df_mice=pd.read_sql_query('SELECT * FROM data LEFT JOIN sites_information on data.sites_information_id = sites_information.id LEFT JOIN coordinates ON sites_information.coordinates_id = coordinates.id LEFT JOIN species_information ON data.species_information_id = species_information.id', conn)
conn.close()

In [ ]:
df_mice = df_mice.drop('id', axis=1)
df_mice["id"] = df_mice.index + 1
print(df_mice.head())

In [ ]:
df_mice.columns

In [ ]:
# Choose only usable data (drop unnecessary data like id)
df_mice = df_mice[["id","event_date","latitude","longitude","scientific_name"]]

In [ ]:
df_mice.head()

In [ ]:
print(f'length of df_mice: {len(df_mice)}')

In [ ]:
df_mice[["scientific_name"]].value_counts()

In [ ]:
# Check NA
df_mice.isna().sum()

# Data Visualisation

## Geo - Map

In [ ]:
# It is not recommended to use pip to install geopandas: https://geopandas.org/en/stable/getting_started/install.html
# Better to use Conda instead of pip
# %conda install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
gdf_mice = gpd.GeoDataFrame(df_mice, 
            geometry=gpd.points_from_xy(df_mice.longitude, df_mice.latitude),
            crs="EPSG:4326" # This line is really important to make coordinate machine readable
)
gdf_mice.drop(columns=['longitude', 'latitude'])

In [ ]:
# import folium

# # # Create a map centered around the mean latitude and longitude
# m = folium.Map(location=[gdf_mice['latitude'].mean(), gdf_mice['longitude'].mean()])

# # # Add the GeoDataFrame to the map as markers
# for idx, row in gdf_mice.iterrows():
#     folium.Marker(location=[row['latitude'], row['longitude']], popup=row['scientific_name']).add_to(m)

# # # Display the map
# m

## Geo - XY-coordinate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 2次元平面上に可視化
df_mice.plot.scatter(x="longitude",y="latitude")

In [ ]:
# マウスの種類別に可視化
sns.relplot(data=df_mice, x="longitude", y="latitude", hue="scientific_name")
plt.show()

→種類が多すぎるので，とりあえずscientific_nameは無視でもいいかも

## Latitude and Longitude

In [ ]:
print(f'latitude_max: {df_mice["latitude"].max()}')
print(f'latitude_min: {df_mice["latitude"].min()}')
print(f'longitude_max: {df_mice["longitude"].max()}')
print(f'longitude_min: {df_mice["longitude"].min()}')

→latitudeとlongitudeの変域
- latitude = [41,42]
- longitude = [11,16]

# Prediction

- 使えそうなデータ
  - ネズミの親子関係（from OpenMICE）
  - 

# Data Analysis

## Gridding of point data with geopandas
Refer to: https://james-brennan.github.io/posts/fast_gridding_geopandas/


In [ ]:
gdf_mice.head()

In [ ]:
world_path = "data/COAS_RG_20M_2016_3035/COAS_RG_20M_2016_3035.shp"
gdf_world = gpd.read_file(world_path, crs="EPSG:3035")
gdf_world.head()

In [ ]:
gdf_world.plot()

In [ ]:
gdf_mice.plot(markersize=10, figsize=(8, 8))

In [ ]:
ax = gdf_mice.plot(markersize=10, figsize=(12, 8), column='scientific_name', cmap='jet')
plt.autoscale(False)
gdf_world.to_crs(gdf_mice.crs).plot(ax=ax, color='none', edgecolor='lightgrey')

In [ ]:
import numpy as np
import shapely
np.random.seed(42) # Don't panic!

## Example of Creating Grid

In [ ]:
# total area for the grid
xmin, ymin, xmax, ymax= gdf_mice.total_bounds
# how many cells across and down
# In this case, cell size is 1/30*(max_latitude - min_latitude)
n_cells=30
cell_size = (xmax-xmin)/n_cells
# projection of the grid
crs = "EPSG:4326"
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs=crs)

In [ ]:
ax = gdf_mice.plot(markersize=1, figsize=(12, 8), column='scientific_name', cmap='jet')
plt.autoscale(False)
cell.plot(ax=ax, facecolor="none", edgecolor='lightgrey')
ax.axis("off")

In [ ]:
gdf_mice_merged = gpd.sjoin(gdf_mice, cell, how='left', predicate='within')

In [ ]:
gdf_mice_merged.head()

In [ ]:
# make a simple count variable that we can sum
gdf_mice_merged['mice_occurrence']=1
# Compute stats per grid cell -- aggregate fires to grid cells with dissolve
dissolve = gdf_mice_merged.dissolve(by="index_right", aggfunc="count")
# put this into cell
cell.loc[dissolve.index, 'mice_occurrence'] = dissolve.mice_occurrence.values

In [ ]:
cell.head()

In [ ]:
cell["mice_occurrence"].max()

In [ ]:
cell["mice_occurrence"].plot(kind='bar')


In [ ]:
ax = cell.plot(column='mice_occurrence', figsize=(12, 8), cmap='viridis', vmax=122, edgecolor="grey", legend=True)
plt.autoscale(False)
plt.colormaps()
gdf_world.to_crs(cell.crs).plot(ax=ax, color='none', edgecolor='lightgrey')
ax.axis('off')

## Flow of Analysis (tentative)
1. gridにid付ける
2. gdf_mice内にgrid_idを入れる
3. grid_idごとにねずみの発生しやすさを求める(should define "発生しやすさ")
4. →gridごとの気温と人口密度（+ねずみの発生を予測しそうな外的要因）を持ってきて，それらの変数からmiceの発生しやすさをモデル化する

miceの発生しやすさ: "likelihood of mice occurrence" (以下"lhmo")
- gridの"lhmo"が大きいほどmiceの報告数が多くなるようにしたい
- 目標: mice_occurrence > 0を元データにして，mice_occurrence = 0のgrid内でのmice_occurrenceの数を予測する
  - holdoutなら，train_data: 80% of total grids (もちろんgrid[mice_occurrence] != 0), test_data: 20% of total gridsみたいな感じ
  - →これで精度を挙げて，grids whose mice_occurrence = 0のmice_occurrenceを予測する

- Problem: 気温や人口のデータがEU圏内，もしくはイタリアでどのような地区単位で収集されているのか知る必要がある．
  - [人口]1km四方のpolygon population dataが存在: https://ec.europa.eu/eurostat/web/gisco/geodata/grids
  - [平均気温]
  - [平均標高]

In [ ]:
# 人口密度メッシュ（5km四方）
gdf_grid_5km = gpd.read_file("data/grid/grid_5km_surf.gpkg")

In [ ]:
gdf_grid_5km = gdf_grid_5km[gdf_grid_5km["CNTR_ID"] == "IT"]

In [ ]:
gdf_grid_5km.head()

In [ ]:
gdf_grid_5km.to_crs("EPSG:4326").plot(column="TOT_P_2021", figsize=(12,8), legend=True, edgecolor="none")
plt.title("Population Density")

In [ ]:
import matplotlib.colors as colors

In [ ]:

ax = gdf_mice.plot(figsize=(12, 8), markersize=0)
plt.title("Mice Occurrence in South-Central Italy")
plt.autoscale(False)
gdf_world.to_crs(gdf_mice.crs).plot(ax=ax, color="grey")
gdf_mice.plot(ax=ax, markersize=8)

In [ ]:
def plot_gdfs(gdf_scatter, gdf_base, lognorm):
    title = ""
    if lognorm:
        norm = colors.LogNorm()
        title = "Population Density (LogNorm)"
    else:
        norm = None
        title = "Population Density"
    ax = gdf_scatter.plot(figsize=(12, 8), markersize=0)
    plt.title(title + " " + "and Mice Occurrence")
    plt.autoscale(False)
    gdf_base.to_crs(gdf_scatter.crs).plot(ax=ax, cmap="gray", column="TOT_P_2021", edgecolor='lightgrey', norm=norm, legend=True)
    gdf_scatter.plot(ax=ax, markersize=10)

In [ ]:
plot_gdfs(gdf_mice,gdf_grid_5km,False)

In [ ]:
# 人口密度が見えにくいのでLogarithmic Normalisationした
## Refer to: https://matplotlib.org/stable/users/explain/colors/colormapnorms.html

plot_gdfs(gdf_mice,gdf_grid_5km,True)

## Flow of Analysis (from now on)
- gdf_grid_5kmに["mice_occurrence", "temperature", "avg_altitude", "precipitation(降水量)","緑化率","耕作地率"]などを突っ込み，grid[mice_occurrence != 0]でモデル作成→モデルをgrid[mice_occurrence = 0]に当てはめてMice_occurrence mapを作成
  1. Gridの中心地点（Grid二対角線の交点）を求めて，gdf_grid_5km["ctr_co"] (="center coordinate")に挿入
  2. Temperature, avg_altitudeは，gridの中心地点（二対角線の交点）に最寄りの（latitude, longitudeの差の二乗和が最も小さい）データを取ってくる
  - Temperatureとavg_altitudeはある程度相関していそうなので，多重共線性（"Multicollinearity", "マルティコ"）に注意

## Think with Grid

In [ ]:
gdf_mice.head()

In [ ]:
gdf_grid_5km.head()

In [ ]:
# Insert "mice_occurrence" (the number of mice observed in the grid) in gdf_grid_5km
gdf_grid_5km_merged = gdf_grid_5km.join(
    gpd.sjoin(gdf_mice.to_crs(gdf_grid_5km.crs), gdf_grid_5km).groupby("index_right").size().rename("mice_occurrence"),
    how="left",
)

In [ ]:
gdf_grid_5km_merged.head()

In [ ]:
gdf_grid_5km_merged["mice_occurrence"].value_counts()

In [ ]:
ax = gdf_grid_5km_merged.plot(column="mice_occurrence", figsize=(12, 8))
plt.title("Population Density (LogNorm)")
plt.autoscale(False)
gdf_grid_5km_merged.plot(ax=ax, cmap="gray", column="TOT_P_2021", edgecolor='lightgrey', norm=colors.LogNorm(), legend=True)
gdf_world.plot(ax=ax, color='none', edgecolor='black')

In [ ]:
ax = gdf_grid_5km_merged.plot(column="mice_occurrence", figsize=(12, 8))
plt.title("Mice Occurrence Density")
plt.autoscale(False)
gdf_grid_5km_merged.plot(ax=ax, cmap="plasma", column="mice_occurrence", edgecolor='lightgrey', legend=True)
gdf_world.plot(ax=ax, color='none', edgecolor='black')

### UAA - Utilised agricultural area (Hectare)

In [ ]:
gdf_agri_uaa = gpd.read_file("data/geospatial_data_from_agricultural_census/c17/c17.shp")

In [ ]:
gdf_agri_uaa.head()

In [ ]:
ax = gdf_grid_5km_merged.plot(column="mice_occurrence", figsize=(12, 8))
plt.autoscale(False)
gdf_agri_uaa.plot(ax=ax,column="UAA", legend=True)

In [ ]:
ax = gdf_grid_5km_merged.plot(column="mice_occurrence", figsize=(12, 8))
plt.autoscale(False)
gdf_agri_uaa.plot(ax=ax,column="UAA", legend=True, norm=colors.LogNorm())

In [ ]:
gdf_grid_5km_uaa = gdf_grid_5km_merged.sjoin(gdf_agri_uaa.to_crs(gdf_grid_5km_merged.crs)[["geometry", "UAA"]], how="inner", predicate='intersects').to_crs("EPSG:4326")

In [ ]:
gdf_grid_5km_merged["geometry"]

In [ ]:
gdf_agri_uaa.to_crs(gdf_grid_5km_merged.crs)["geometry"]

In [ ]:
gdf_grid_5km_uaa["UAA"].value_counts()

In [ ]:
gdf_grid_5km_uaa["mice_occurrence"].value_counts()

In [ ]:
gdf_grid_5km_uaa.plot(column="UAA", norm=colors.LogNorm(), figsize=(12,8), legend=True)
plt.title("UAA")

In [ ]:
gdf_grid_5km_uaa

## Handling .nc file
Refer to: https://spatial-dev.guru/2024/01/20/get-the-geographical-coordinates-from-netcdf-file-using-python/

In [ ]:
# %pip install rioxarray

In [ ]:
import xarray as xr
import math

In [ ]:
def convert_nc_to_gdf(nc_path, shp_path):
    ds = xr.open_dataset(nc_path, decode_coords="all")
    x, y = np.meshgrid(ds.longitude, ds.latitude)
    variables = list(ds.var())
    variable_name = variables[0]
    raster_values = ds[variable_name][0].values

    df = pd.DataFrame({
    "x": x.flatten(),
    "y": y.flatten(),
    "raster_value": raster_values.flatten()
    })
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']))
    gdf[~gdf["raster_value"].apply(lambda x: math.isnan(x))].drop(columns=["x","y"]).rename(columns={"raster_value": variable_name}).to_file(shp_path)

### Mean temperature (0.25)

In [ ]:
convert_nc_to_gdf("data/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e.nc",
                  "data/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e.shp")

In [ ]:
gdf_mean_temp = gpd.read_file("data/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e/tg_ens_mean_0.25deg_reg_2011-2024_v30.0e.shp")

In [ ]:
gdf_mean_temp = gdf_mean_temp.set_crs("EPSG:4326")

In [ ]:
gdf_mean_temp.head()

In [ ]:
gdf_mean_temp.plot(column="tg", legend=True)
plt.title("gdf_mean_temp")

### Precipitation (0.25)

In [ ]:
convert_nc_to_gdf("data/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e.nc",
                  "data/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e.shp")

In [ ]:
gdf_precip = gpd.read_file("data/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e/rr_ens_mean_0.25deg_reg_2011-2024_v30.0e.shp")

In [ ]:
gdf_precip = gdf_precip.set_crs("EPSG:4326")

In [ ]:
gdf_precip.head()

In [ ]:
gdf_precip.plot(column="rr", legend=True)
plt.title("gdf_precip")

In [ ]:
gdf_precip.rr.value_counts()

## Merge with GDF_Grid_5km

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_5km_uaa.plot(ax=ax, column="UAA", facecolor="none", edgecolor="lightgrey")
gdf_precip.plot(ax=ax, column="rr", legend=True)

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_5km_uaa.plot(ax=ax, column="UAA", facecolor="none", edgecolor="lightgrey")
gdf_mean_temp.plot(ax=ax, column="tg", legend=True)

→気温．降水量の粒度低い！0.25degではなく0.1degでやろう

### Mean temperature & Precipitation(0.1)

In [ ]:
convert_nc_to_gdf("data/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc",
                  "data/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e.shp")

In [ ]:
convert_nc_to_gdf("data/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc",
                  "data/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e.shp")

In [ ]:
gdf_mean_temp = gpd.read_file("data/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e/tg_ens_mean_0.1deg_reg_2011-2024_v30.0e.shp").set_crs("EPSG:4326")
gdf_precip = gpd.read_file("data/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e/rr_ens_mean_0.1deg_reg_2011-2024_v30.0e.shp").set_crs("EPSG:4326")

In [ ]:
gdf_precip.plot(column="rr")

In [ ]:
gdf_precip.plot(column="rr",norm=colors.CenteredNorm())

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_5km_uaa.plot(ax=ax, column="UAA", facecolor="none", edgecolor="lightgrey")
gdf_precip.plot(ax=ax, column="rr", legend=True, markersize=1)

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_5km_uaa.plot(ax=ax, column="UAA", facecolor="none", edgecolor="lightgrey")
gdf_mean_temp.plot(ax=ax, column="tg", legend=True, markersize=1)

# 10km四方か20km四方の方がbetter?

In [ ]:
# 人口密度メッシュ（10km四方）
gdf_grid_10km = gpd.read_file("data/grid/grid_10km_surf.gpkg")
gdf_grid_10km = gdf_grid_10km[gdf_grid_10km["CNTR_ID"] == "IT"].to_crs("EPSG:4326")

In [ ]:
# 人口密度メッシュ（20km四方）
gdf_grid_20km = gpd.read_file("data/grid/grid_20km_surf.gpkg")
gdf_grid_20km = gdf_grid_20km[gdf_grid_20km["CNTR_ID"] == "IT"].to_crs("EPSG:4326")

In [ ]:
plot_gdfs(gdf_mice,gdf_grid_10km,True)

In [ ]:
plot_gdfs(gdf_mice,gdf_grid_20km,True)

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_10km.plot(ax=ax, facecolor="none", edgecolor="lightgrey")
gdf_mean_temp.plot(ax=ax, column="tg", legend=True, markersize=1)

In [ ]:
ax = gdf_mice.plot(markersize=0, figsize=(12,8))
plt.autoscale(False)
gdf_grid_20km.plot(ax=ax, facecolor="none", edgecolor="lightgrey")
gdf_mean_temp.plot(ax=ax, column="tg", legend=True, markersize=1)

## →10km四方で行く

## ここからの流れ
1. 10km四方で気温，降水量，ねずみoccurrence，人口密度を突っ込んだgdf作る
2. neural networkで予測モデル作る: outputは連続値　(train_data=gdf[gdf[ねずみoccurrence]!=0])
3. Cross validationなどでテスト
4. 最終的に，gdf[gdf[ねずみoccurrence]=0]にも連続値を当てはめてみて，ネズミの発生分布をつくる
5. レポートにする！

In [ ]:
# Insert "mice_occurrence" (the number of mice observed in the grid) in gdf_grid_5km
gdf_grid_10km_merged = gdf_grid_10km.join(
    gpd.sjoin(gdf_mice.to_crs(gdf_grid_10km.crs), gdf_grid_10km).groupby("index_right").size().rename("mice_occurrence"),
    how="left",
)

In [ ]:
gdf_grid_10km_merged["mice_occurrence"].value_counts()

In [ ]:
gdf_grid_10km_uaa = gdf_grid_10km_merged.sjoin(gdf_agri_uaa.to_crs(gdf_grid_10km_merged.crs)[["geometry", "UAA"]], how="inner", predicate='intersects').to_crs("EPSG:4326")
gdf_grid_10km_uaa = gdf_grid_10km_uaa.drop(["index_right"],axis=1)

In [ ]:
gdf_grid_10km_uaa_precip = gdf_grid_10km_uaa.sjoin(gdf_precip[["geometry", "rr"]], how="inner", predicate='intersects').to_crs("EPSG:4326")
gdf_grid_10km_uaa_precip = gdf_grid_10km_uaa_precip.drop(["index_right"],axis=1)

In [ ]:
gdf_grid_10km_uaa_precip_temp_all = gdf_grid_10km_uaa_precip.sjoin(gdf_mean_temp[["geometry", "tg"]], how="inner", predicate='intersects').to_crs("EPSG:4326")
gdf_grid_10km_uaa_precip_temp_all = gdf_grid_10km_uaa_precip_temp_all.drop(["index_right"],axis=1)

In [ ]:
gdf_grid_10km_uaa_precip_temp = gdf_grid_10km_uaa_precip_temp_all[["geometry","mice_occurrence","TOT_P_2021","UAA","rr","tg"]]

In [ ]:
gdf_grid_10km_uaa_precip_temp[["rr","tg"]].value_counts()

## Fill NaN

In [ ]:
gdf_grid_10km_uaa_precip_temp["mice_occurrence"].value_counts()

In [ ]:
len(gdf_grid_10km_uaa_precip_temp)

In [ ]:
gdf_grid_10km_uaa_precip_temp[["TOT_P_2021","mice_occurrence","rr","tg"]].isna().sum()

→最高！

In [ ]:
gdf_grid_10km_uaa_precip_temp[["TOT_P_2021","mice_occurrence","rr","tg"]].isna().sum()

## Visualisation

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(facecolor="none", figsize=(12,8))
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="mice_occurrence", legend=True, figsize=(12,8))
plt.title("Mice Occurrence")

In [ ]:
gdf_grid_10km_uaa_precip_temp.plot(column="tg", legend=True, figsize=(12,8))
plt.title("mean temperature")

In [ ]:
gdf_grid_10km_uaa_precip_temp.plot(column="rr", legend=True, figsize=(12,8))
plt.title("precipitation")

In [ ]:
gdf_grid_10km_uaa_precip_temp.plot(column="rr", legend=True, figsize=(12,8), norm=colors.LogNorm())
plt.title("precipitation LogNorm")

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", figsize=(12,8), facecolor="none")
plt.autoscale(False)
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="mice_occurrence", legend=True)
plt.title("mice_occurrence")

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", figsize=(12,8), facecolor="none")
plt.autoscale(False)
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="tg", legend=True)
plt.title("mean temperature")

→地球の表面の丸み（？）によって縞模様が出来てる？
縞模様は欠損値？

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", figsize=(12,8), facecolor="none")
plt.autoscale(False)
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="rr", legend=True)
plt.title("precipitation")

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", figsize=(12,8), facecolor="none")
plt.autoscale(False)
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="rr", legend=True, norm=colors.LogNorm())
plt.title("precipitation (LogNorm)")

In [ ]:
ax = gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", figsize=(12,8), facecolor="none")
plt.autoscale(False)
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="TOT_P_2021", legend=True)
plt.title("Total Population (2021)")

In [ ]:
gdf_grid_10km_uaa_precip_temp.head()

In [ ]:
gdf_grid_10km_uaa_precip_temp.dtypes


→geometry（id扱い）以外は全て数値データになった！ Ready to analyse

# Machine Learning
やっと解析に入れる！！！！歓喜

In [ ]:
gdf_grid_10km_uaa_precip_temp

In [ ]:
# train dataを目的変数X，説明変数Yに分ける
train_X_geom = gdf_grid_10km_uaa_precip_temp[~gdf_grid_10km_uaa_precip_temp["mice_occurrence"].isnull()].drop("mice_occurrence",axis=1).reset_index(drop=True)
train_y_geom = gdf_grid_10km_uaa_precip_temp[~gdf_grid_10km_uaa_precip_temp["mice_occurrence"].isnull()]["mice_occurrence"].reset_index(drop=True)

In [ ]:
# mice_occurrence = Nanのものをtest_Xへ
test_X_geom = gdf_grid_10km_uaa_precip_temp[gdf_grid_10km_uaa_precip_temp["mice_occurrence"].isnull()].drop("mice_occurrence",axis=1).reset_index(drop=True)

In [ ]:
X_test_geom = test_X_geom[["geometry"]].reset_index(drop=True)
X_test = test_X_geom.drop("geometry",axis=1).reset_index(drop=True)

## LightGBM

### Hold-out

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
# train dataの内20%をtest dataへ
X_train_and_geom, X_valid_and_geom, y_train, y_valid = train_test_split(train_X_geom, train_y_geom, test_size=0.2, random_state=42)

In [ ]:
X_train_geom = X_train_and_geom[["geometry"]].reset_index(drop=True)
X_train = X_train_and_geom.drop("geometry", axis=1).reset_index(drop=True)

In [ ]:
X_valid_geom = X_valid_and_geom[["geometry"]].reset_index(drop=True)
X_valid = X_valid_and_geom.drop("geometry", axis=1).reset_index(drop=True)

In [ ]:
len(X_train)

In [ ]:
len(X_valid)

In [ ]:
categories = ["TOT_P_2021","UAA","rr","tg"]

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categories, free_raw_data=False)
lgb_eval = lgb.Dataset(X_valid, y_valid, categorical_feature=categories, free_raw_data=False, reference=lgb_train)

In [ ]:
lgbm_params = {
    "early_stopping_rounds": 20,
    "seed": 42 # Don't Panic!
}

In [ ]:
model_lgb = lgb.train(lgbm_params,
                      lgb_train,
                      valid_sets=lgb_eval,
                      num_boost_round=100)

In [ ]:
model_lgb.feature_importance()

In [ ]:
importance = pd.DataFrame(model_lgb.feature_importance(), index=X_train.columns, columns=["importance"]).sort_values(by="importance",ascending=True)
importance.plot.barh()

- そもそも全然雨が降ってないから降水量が効かない
- Populationが効いているのは，そもそも人口が多いところの方がネズミが報告されやすいというデータの特性もありそう

In [ ]:
y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)

In [ ]:
df_y_pred = pd.DataFrame(y_pred, columns=["pred_mice_occurrence"])

In [ ]:
df_y_pred_geom = pd.concat([df_y_pred, X_valid_geom], axis=1)

In [ ]:
gdf_y_pred_geom = gpd.GeoDataFrame(df_y_pred_geom)

In [ ]:
gdf_y_pred_geom = gdf_y_pred_geom.set_crs("EPSG:4326")

In [ ]:
gdf_y_pred_geom

In [ ]:
gdf_grid_10km_uaa_precip_temp[gdf_grid_10km_uaa_precip_temp["mice_occurrence"]>0]

In [ ]:
gdf_grid_10km_uaa_precip_temp.plot(column="mice_occurrence", legend=True)
plt.title("pre-trained mice_occurrence")

In [ ]:
gdf_y_pred_geom.plot(column="pred_mice_occurrence", legend=True)
plt.title("predicted mice_occurrence")

In [ ]:
y_pred_whole_italy = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
df_y_pred_whole_italy = pd.DataFrame(y_pred_whole_italy, columns=["pred_mice_occurrence"])
df_y_pred_whole_italy_geom = pd.concat([df_y_pred_whole_italy, X_test_geom], axis=1)
gdf_y_pred_whole_italy_geom = gpd.GeoDataFrame(df_y_pred_whole_italy_geom)
gdf_y_pred_whole_italy_geom = gdf_y_pred_whole_italy_geom.set_crs("EPSG:4326")

In [ ]:
gdf_y_pred_whole_italy_geom.plot(column="pred_mice_occurrence", legend=True, figsize=(12,8))
plt.title("predicted mice_occurrence in whole italy")

In [ ]:
ax = gdf_y_pred_whole_italy_geom.plot(column="pred_mice_occurrence", legend=True, figsize=(12,8), vmin=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].min(), vmax=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].max())
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="mice_occurrence", vmin=gdf_y_pred_whole_italy_geom["pred_mice_occurrence"].min(), vmax=gdf_y_pred_whole_italy_geom["pred_mice_occurrence"].max())
plt.title("predicted+real mice_occurrence in whole italy")

In [ ]:
gdf_y_pred_whole_italy_geom.plot(column="pred_mice_occurrence", legend=True, figsize=(12,8), vmin=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].min(), vmax=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].max())
plt.title("predicted mice_occurrence in whole italy")

In [ ]:
ax = gdf_y_pred_whole_italy_geom.plot(column="pred_mice_occurrence", legend=True, figsize=(12,8), vmin=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].min(), vmax=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].max())
gdf_grid_10km_uaa_precip_temp.plot(ax=ax, column="mice_occurrence", vmin=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].min(), vmax=gdf_grid_10km_uaa_precip_temp["mice_occurrence"].max())
plt.title("predicted+real mice_occurrence in whole italy")

### Hold-out Analyse Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_valid, np.round(y_pred))

とりあえず結果と言える結果になった！
## TODO
- 縞模様になっている欠損gridの原因を突き止める: 規則的になっているので，気象データを読み込む時にこうなった可能性がある．
- とりあえずLightGBM以外を試すには時間が無いから，cross validationで精度が上がるかだけ見てみる？
  - →もち，他の分析手法を試すのもあり
- 論文の形にする